In [1]:
!pip install nltk

In [2]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to C:\Users\AULIA
[nltk_data]     ASUSK401L\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\AULIA
[nltk_data]     ASUSK401L\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\AULIA
[nltk_data]     ASUSK401L\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\AULIA
[nltk_data]     ASUSK401L\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
import pandas as pd
from flair.models import TextClassifier
from flair.data import Sentence
sia = TextClassifier.load('en-sentiment')

In [ ]:
kaggel_data = pd.read_csv('tweet_kaggle.csv')
scrap_data = pd.read_csv('tweets_crawl_new.csv')
data_file = kaggel_data.append(scrap_data)

In [5]:
data_file

,twitid,twitcontent,username
0,1467810369,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",_TheSpecialOne_
1,1467810672,is upset that he can't update his Facebook by ...,scotthamilton
2,1467810917,@Kenichan I dived many times for the ball. Man...,mattycus
3,1467811184,my whole body feels itchy and like its on fire,ElleCTF
4,1467811193,"@nationwideclass no, it's not behaving at all....",Karoli
...,...,...,...
1795,1597688327142281216,@ChefTomKerridge commentating on the england w...,teejayflaherty1
1796,1597688326164992000,"The Welsh passion, the pride, the fight, wow.....",KevRussell7
1797,1597688324634087425,It is incredible to see how #ai failing to pre...,Icocbekristos
1798,1597688322272509952,Im running out of superlatives for Ream as a C...,shaneevent11


In [6]:
data_file = data_file.drop_duplicates('twitcontent')

In [7]:
data_file.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2042 entries, 0 to 1798
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   twitid       2042 non-null   int64 
 1   twitcontent  2042 non-null   object
 2   username     2042 non-null   object
dtypes: int64(1), object(2)
memory usage: 63.8+ KB


In [8]:
import re
import string

In [ ]:
def clean_text(text):
    
    text = str(text).lower()
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    
    return text
data_file['twitcontent'] = data_file['twitcontent'].apply(lambda x:clean_text(x))

In [ ]:
df=pd.DataFrame()
data_file['twitcontent']=data_file['twitcontent']
def tokenization(text):
    text = re.split('\W+', text)
    return text

data_file['tokenized'] = data_file['twitcontent'].apply(lambda x: tokenization(x.lower()))
stopword = nltk.corpus.stopwords.words('english')
def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text
    
data_file['No_stopwords'] = data_file['tokenized'].apply(lambda x: remove_stopwords(x))

ps = nltk.PorterStemmer()

def stemming1(text):
    text = [ps.stem(word) for word in text]
    return text

data_file['stemmed_porter'] = data_file['No_stopwords'].apply(lambda x: stemming1(x))

from nltk.stem.snowball import SnowballStemmer
s_stemmer = SnowballStemmer(language='english')
def stemming2(text):
    text = [s_stemmer.stem(word) for word in text]
    return text
data_file['stemmed_snowball'] = data_file['No_stopwords'].apply(lambda x: stemming2(x))

wn = nltk.WordNetLemmatizer()

def lemmatizer(text):
    text = [wn.lemmatize(word) for word in text]
    return text

data_file['lemmatized'] = data_file['No_stopwords'].apply(lambda x: lemmatizer(x))

In [37]:
import flair

In [38]:
sentiment_model = flair.models.TextClassifier.load('en-sentiment')

2022-12-01 00:27:01,177 loading file C:\Users\AULIA ASUSK401L\.flair\models\sentiment-en-mix-distillbert_4.pt


In [39]:
sentiment = []
confidence = []

for sentence in data_file['twitcontent']:
    if sentence == "":
        sentiment.append("")
        confidence.append("")
    
    else:
        sample = flair.data.Sentence(sentence)
        sentiment_model.predict(sample)
        
        sentiment.append(sample.labels[0].value)
        confidence.append(sample.labels[0].score)

In [ ]:
data_file['sentiment'] = sentiment
data_file['confidence'] = confidence

In [41]:
data_file

,twitid,twitcontent,username,tokenized,No_stopwords,stemmed_porter,stemmed_snowball,lemmatized,sentiment_flair,sentiment,confidence
0,1467810369,switchfoot httptwitpiccom2y1zl awww thats a b...,_TheSpecialOne_,"[switchfoot, httptwitpiccom2y1zl, awww, thats,...","[switchfoot, httptwitpiccom2y1zl, awww, thats,...","[switchfoot, httptwitpiccom2y1zl, awww, that, ...","[switchfoot, httptwitpiccom2y1zl, awww, that, ...","[switchfoot, httptwitpiccom2y1zl, awww, thats,...",negative,NEGATIVE,0.996120
1,1467810672,is upset that he cant update his facebook by t...,scotthamilton,"[is, upset, that, he, cant, update, his, faceb...","[upset, cant, update, facebook, texting, might...","[upset, cant, updat, facebook, text, might, cr...","[upset, cant, updat, facebook, text, might, cr...","[upset, cant, update, facebook, texting, might...",negative,NEGATIVE,0.996055
2,1467810917,kenichan i dived many times for the ball manag...,mattycus,"[kenichan, i, dived, many, times, for, the, ba...","[kenichan, dived, many, times, ball, managed, ...","[kenichan, dive, mani, time, ball, manag, save...","[kenichan, dive, mani, time, ball, manag, save...","[kenichan, dived, many, time, ball, managed, s...",positive,POSITIVE,0.854769
3,1467811184,my whole body feels itchy and like its on fire,ElleCTF,"[my, whole, body, feels, itchy, and, like, its...","[whole, body, feels, itchy, like, fire, ]","[whole, bodi, feel, itchi, like, fire, ]","[whole, bodi, feel, itchi, like, fire, ]","[whole, body, feel, itchy, like, fire, ]",negative,NEGATIVE,0.985915
4,1467811193,nationwideclass no its not behaving at all im ...,Karoli,"[nationwideclass, no, its, not, behaving, at, ...","[nationwideclass, behaving, im, mad, cant, see, ]","[nationwideclass, behav, im, mad, cant, see, ]","[nationwideclass, behav, im, mad, cant, see, ]","[nationwideclass, behaving, im, mad, cant, see, ]",negative,NEGATIVE,0.999555
...,...,...,...,...,...,...,...,...,...,...,...
1794,1597688328098557952,lets go usa worldcup,Void_Chasers,"[lets, go, usa, worldcup, ]","[lets, go, usa, worldcup, ]","[let, go, usa, worldcup, ]","[let, go, usa, worldcup, ]","[let, go, usa, worldcup, ]",positive,POSITIVE,0.978026
1795,1597688327142281216,cheftomkerridge commentating on the england wa...,teejayflaherty1,"[cheftomkerridge, commentating, on, the, engla...","[cheftomkerridge, commentating, england, wales...","[cheftomkerridg, comment, england, wale, game,...","[cheftomkerridg, comment, england, wale, game,...","[cheftomkerridge, commentating, england, wale,...",positive,POSITIVE,0.994031
1796,1597688326164992000,the welsh passion the pride the fight wowits a...,KevRussell7,"[the, welsh, passion, the, pride, the, fight, ...","[welsh, passion, pride, fight, wowits, shame, ...","[welsh, passion, pride, fight, wowit, shame, t...","[welsh, passion, pride, fight, wowit, shame, t...","[welsh, passion, pride, fight, wowits, shame, ...",negative,NEGATIVE,0.865182
1797,1597688324634087425,it is incredible to see how ai failing to pred...,Icocbekristos,"[it, is, incredible, to, see, how, ai, failing...","[incredible, see, ai, failing, predict, worldc...","[incred, see, ai, fail, predict, worldcup, win...","[incred, see, ai, fail, predict, worldcup, win...","[incredible, see, ai, failing, predict, worldc...",negative,NEGATIVE,0.884833


In [48]:
!pip install streamlit

  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19582 sha256=7ae14ab2de866c7949ad69c991d99fc9aef4d409189c9cde87cba4345b81d4d2
  Stored in directory: c:\users\aulia asusk401l\appdata\local\pip\cache\wheels\2d\f0\a8\1094fca7a7e5d0d12ff56e0c64675d72aa5cc81a5fc200e849
Successfully built validators


In [50]:
import streamlit as st

In [ ]:
temp = data_file.groupby('sentiment_flair').count()['twitcontent'].reset_index().sort_values(by='twitcontent',ascending=False)
temp.style.background_gradient(cmap='Purples')

,sentiment_flair,twitcontent
0,negative,1148
1,positive,894


In [51]:
data_file.to_csv(r'data\data_file.csv', index = False)

In [ ]:
!pip --default-timeout=1200 install atoti[jupyterlab]

In [33]:
import atoti as tt

session = tt.Session(user_content_storage='./data')

Welcome to atoti 0.7.2!

By using this community edition, you agree with the license available at https://docs.atoti.io/latest/eula.html.
Browse the official documentation at https://docs.atoti.io.
Join the community at https://www.atoti.io/register.

atoti collects telemetry data, which is used to help understand how to improve the product.
If you don't wish to send usage data, set the ATOTI_DISABLE_TELEMETRY environment variable to True.

You can hide this message by setting the ATOTI_HIDE_EULA_MESSAGE environment variable to True.


In [52]:
table = session.read_csv("data/data/data_file.csv", keys=["twitid"])

AtotiJavaException: LocalPath must be absolute.